In [1]:
import os
from typing import List

import requests as requests
from typing import Union

import datetime
from utils.eth_blocks_utils import get_block_for_timestamp

In [2]:
def alchemy_asset_transfers(
        from_address: str,
        to_address: str,
        from_block: Union[int, str],
        to_block: Union[int, str] = 'latest',
        page_key: str = None
):
    if not to_block == 'latest':
        to_block = hex(to_block)

    parameters = {
        "fromBlock": hex(from_block),
        "toBlock": to_block,
        "fromAddress": from_address,
        "toAddress": to_address
    }

    if not from_address:
        parameters.pop('fromAddress')
    if not to_address:
        parameters.pop('toAddress')

    if page_key:
        parameters['pageKey'] = page_key

    response_post = requests.post(
        "https://eth-mainnet.alchemyapi.io/v2/"
        + os.environ["ALCHEMY_API_KEY"],
        json={
            "jsonrpc": "2.0",
            "id": 0,
            "method": "alchemy_getAssetTransfers",
            "params": [parameters]
        },
    )

    response_json = response_post.json()
    return response_json

def get_lp_swaps(
        from_address: str,
        to_address: str,
        from_block: Union[int, str],
        to_block: Union[int, str]
) -> List:

    page_key = None
    all_txes = []
    while True:
        response = alchemy_asset_transfers(
            from_address=from_address,
            to_address=to_address,
            from_block=from_block,
            to_block=to_block,
            page_key=page_key
        )
        transactions = response["result"]["transfers"]
        all_txes.extend(transactions)
        if not 'pageKey' in response['result'].keys():
            break
        page_key = response['result']['pageKey']

    return all_txes

In [99]:
pool_addr = "0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7"

In [103]:
fraction_fee = 0.03 / 100

In [ ]:
current_block =

In [ ]:
blocks = list(
    range(
        int(get_block_for_timestamp(datetime.datetime.ut)),

    )
)

In [115]:
block_ranges = list(zip(blocks, blocks[1:]))
block_ranges

[(12693255, 12733780),
 (12733780, 12740342),
 (12740342, 12826286),
 (12826286, 12865685),
 (12865685, 12960904),
 (12960904, 12993519),
 (12993519, 13048460),
 (13048460, 13073420),
 (13073420, 13182518),
 (13182518, 13206754),
 (13206754, 13278344),
 (13278344, 13307602),
 (13307602, 13360861),
 (13360861, 13402155),
 (13402155, 13456217),
 (13456217, 13484749),
 (13484749, 13541142)]

In [145]:
fees_for_range = {}
for (block_start, block_end) in block_ranges:

    from_txes = get_lp_swaps(
        from_address=pool_addr, to_address='', from_block=block_start, to_block=block_end
    )
    from_tx_hashes = [i['hash'] for i in from_txes]

    to_txes = get_lp_swaps(
        from_address='', to_address=pool_addr, from_block=block_start, to_block=block_end
    )
    to_tx_hashes = [i['hash'] for i in to_txes]

    total_fees = 0
    for i in to_txes:
        # add_liquidity txes and swaps
        out_value = i['value']
        total_fees += i['value'] * fraction_fee

    for i in from_txes:
        if i['hash'] not in to_tx_hashes:
            # here very rough estimates of fees for removing liquidity
            total_fees += i['value'] * fraction_fee
            in_value = out_value / (1 - fraction_fee)
            total_fees += in_value * fraction_fee
            print(i['hash'])
            raise

    fees_for_range[(block_start, block_end)] = total_fees

0x57d5cfcf1a30f6f756a5fcaf55835250fab0c703b8ebeb280898869cd8cc6562


RuntimeError: No active exception to reraise

In [144]:
i['hash']

'0xb3cf211ce2cf3c92e88d9f18322562736f59ee995abf649cdb44f37f2fcb4986'

In [136]:
fees_for_range

{(12693255, 12733780): 286153.683618792,
 (12733780, 12740342): 86525.73740506361,
 (12740342, 12826286): 769659.7330381555,
 (12826286, 12865685): 377819.2952715634,
 (12865685, 12960904): 884789.9911307511,
 (12960904, 12993519): 309939.782394324,
 (12993519, 13048460): 545952.9301074691,
 (13048460, 13073420): 282702.77075993945,
 (13073420, 13182518): 877265.8728324103,
 (13182518, 13206754): 303965.9061378624,
 (13206754, 13278344): 692746.6687058664,
 (13278344, 13307602): 324502.46422951383,
 (13307602, 13360861): 376507.43964128953,
 (13360861, 13402155): 565520.4336126104,
 (13402155, 13456217): 858646.3213057646,
 (13456217, 13484749): 283132.3856365297,
 (13484749, 13541142): 745315.8085302708}

In [121]:
fees_for_range

{(12693255, 12733780): 235315.84329081108,
 (12733780, 12740342): 51611.64508063634,
 (12740342, 12826286): 486920.3576488525,
 (12826286, 12865685): 275079.7803891132,
 (12865685, 12960904): 741996.076269439,
 (12960904, 12993519): 232767.91744170382,
 (12993519, 13048460): 379169.4613490703,
 (13048460, 13073420): 199588.39389553465,
 (13073420, 13182518): 574380.8313389953,
 (13182518, 13206754): 145389.58763483487,
 (13206754, 13278344): 479242.510586363,
 (13278344, 13307602): 252981.9117894561,
 (13307602, 13360861): 327884.333095976,
 (13360861, 13402155): 312335.41524615674,
 (13402155, 13456217): 569665.7341419263,
 (13456217, 13484749): 227896.96185654632,
 (13484749, 13541142): 645644.3463880663}